## 自动求导

假设我们想对函数$y = 2 \pmb x^\top \pmb x$ 关于列向量$\pmb x$求导

In [2]:
import torch
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

### 在我们计算$y$关于$\pmb x$的梯度之前，我们需要一个地方存储梯度。

In [5]:
# 等价与x = torch.arange(4.0, requires_grad=True)
x.requires_grad_(True)
x.grad # 默认值是None

### 计算$y$

In [6]:
# 内积
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

### 通过调用反向传播函数来自动计算$y$关于$x$每个分量的梯度

In [7]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [8]:
x.grad == 4 * x

tensor([True, True, True, True])

### 计算$x$的另一个函数

In [9]:
# 默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

其中，向量$\pmb x$求和相当于向量$\pmb x$乘一个单位向量$\pmb e$，那么，$y$对$x$求导后，$y^{'} = \pmb e$ 

### 在深度学习中，我们的目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和。 

In [10]:
# 对非标量调用`backward`需要传入一个`gradient`参数
x.grad.zero_()
y = x * x
# 等
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

### 将某些计算移动到记录的计算图之外

In [18]:
x.grad.zero_()
y = x * x
# 当前y是一个函数，通过detach将y变为一个常数赋值给u， 是一个常数
u = y.detach()
# z关于x的导数便是一个 `常数 * x`
z = u * x
# z.sum()
z.sum().backward()
x.grad == u, z.sum()

(tensor([True, True, True, True]), tensor(36., grad_fn=<SumBackward0>))

In [19]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

### 即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍可以计算得到变量的梯度

In [22]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()
a.grad == d/a

tensor(True)